# CHOWDER

## Imports

In [1]:
!pip install torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 5.0 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset, ConcatDataset
import torch.nn as nn
import torch.optim as optim
import torchmetrics

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Load Data

In [4]:
processed_data_path = '/content/drive/My Drive/Breast_Cancer_Detection/Processed_Data/'

X_dev = np.load(processed_data_path + 'X_dev.npy')
y_dev = np.load(processed_data_path + 'y_dev.npy')
X_zoom = np.load(processed_data_path + 'zoom_train.npy')
X_coordinates = np.load(processed_data_path + 'coordinates_dev.npy')

X_test = np.load(processed_data_path + 'X_test.npy')
X_zoom_test = np.load(processed_data_path + 'zoom_test.npy')
X_coordinates_test = np.load(processed_data_path + 'coordinates_test.npy')

In [5]:
X_development = np.concatenate((X_dev, X_zoom, X_coordinates), axis=1)
X_test = np.concatenate((X_test, X_zoom_test, X_coordinates_test), axis=1)

moco_features = X_dev.shape[1]

In [6]:
# Free Memory
del X_dev
del X_zoom
del X_coordinates
del X_zoom_test
del X_coordinates_test

## Split Data

In [7]:
X_train, X_val, y_train, y_val = train_test_split(X_development, y_dev, test_size=0.15, stratify=y_dev, random_state=42)

In [8]:
# Free up memory
del X_development
del y_dev

## Standardize Data

In [9]:
def X_standardize(X_train, X_val, X_test, moco_features):

    # MoCo Features
    moco_train = X_train[:, :moco_features]
    moco_val = X_val[:, :moco_features]
    moco_test = X_test[:, :moco_features]

    moco_mean = np.mean(moco_train)
    moco_std = np.std(moco_train)

    moco_train_scaled = (moco_train - moco_mean) / moco_std
    moco_val_scaled = (moco_val - moco_mean) / moco_std
    moco_test_scaled = (moco_test - moco_mean) / moco_std

    del moco_train
    del moco_val
    del moco_test

    # Zoom
    zoom_train = X_train[:, moco_features:moco_features+1000]
    zoom_val = X_val[:, moco_features:moco_features+1000]
    zoom_test = X_test[:, moco_features:moco_features+1000]

    zoom_mean = np.mean(zoom_train)
    zoom_std = np.std(zoom_train)

    zoom_train_scaled = (zoom_train - zoom_mean) / zoom_std
    zoom_val_scaled = (zoom_val - zoom_mean) / zoom_std
    zoom_test_scaled = (zoom_test - zoom_mean) / zoom_std

    del zoom_train
    del zoom_val
    del zoom_test

    # Merge Data
    X_train_scaled = np.concatenate((moco_train_scaled, zoom_train_scaled, X_train[:, moco_features+1000:]), axis=1)
    X_val_scaled = np.concatenate((moco_val_scaled, zoom_val_scaled, X_val[:, moco_features+1000:]), axis=1)
    X_test_scaled = np.concatenate((moco_test_scaled, zoom_test_scaled, X_test[:, moco_features+1000:]), axis=1)

    return X_train_scaled, X_val_scaled, X_test_scaled

In [10]:
X_train_scaled, X_val_scaled, X_test_scaled = X_standardize(X_train, X_val, X_test, moco_features)

In [11]:
# Free Memory
del X_train
del X_val
del X_test

## Convert to Tensor

In [12]:
X_train_tensor = torch.Tensor(X_train_scaled)
X_val_tensor = torch.Tensor(X_val_scaled)
X_test_tensor = torch.Tensor(X_test_scaled)

In [13]:
# Delete redundant variables to free up memory
del X_test_scaled
del X_train_scaled
del X_val_scaled

## CHOWDER Model

In [14]:
class CHOWDER(nn.Module):

    def __init__(self):
        super(CHOWDER, self).__init__()

        # Convolutional layer
        self.conv = nn.Conv1d(in_channels=1, out_channels=1, kernel_size=2048, stride=2048)

        self.fc1 = nn.Linear(5, 1)

        self.init_weights()

    def init_weights(self):

        # Set std to the square root of the number of edges
        std_conv = 2048**(-0.5)
        # Initialize weights with random normal values for the convolutional layer
        nn.init.normal_(self.conv.weight, mean=0.0, std=std_conv)
        nn.init.constant_(self.conv.bias, 0)


    def forward(self, x):

        # Seperate x into moco data and metadata
        X_moco = x[:, :, :2048000]
        X_zoom = x[:, :, 2048000:2048000+1000]
        X_coord = x[:, :, 2048000+1000:]


        ## CONVOLUTION LAYER

        conv_output = self.conv(X_moco)

        # Calculate L2-norm on weights in the convolutional layer
        l2_reg_conv = 0.0
        for param in self.conv.parameters():
            l2_reg_conv += torch.sum(param ** 2)


        ## MINMAX LAYER

        # Sort each row of the conv layer (each row is a sample)
        sorted_output, sorted_indices = torch.sort(conv_output, dim=2)

        # Number of top instances and negative evidence
        R = 2

        # Select the first two and last two sorted tiles
        selected_output_top = sorted_output[:, :, :R]
        selected_output_bottom = sorted_output[:, :, :-2]
        selected_tiles = torch.cat((selected_output_top, selected_output_bottom[:, :, -R:]), dim=2)

        # Coordinate
        selected_indices = sorted_indices[:, :, :R]
        selected_coord_top_1 = torch.gather(X_coord, 2, 2*selected_indices)
        selected_coord_top_2 = torch.gather(X_coord, 2, 2*selected_indices+1)

        # Calculate the differences in x and y coordinates
        x_diff = selected_coord_top_1[:, :, 1:] - selected_coord_top_1[:, :, :-1]
        y_diff = selected_coord_top_2[:, :, 1:] - selected_coord_top_2[:, :, :-1]

        # Square the differences
        x_diff_squared = x_diff**2
        y_diff_squared = y_diff**2

        # Sum the squared differences along the last dimension
        sum_squared_diff = x_diff_squared + y_diff_squared

        # Take the square root to get the distance
        top_distance = torch.sqrt(sum_squared_diff)


        # Join tiles and zoom
        a0 = torch.cat((selected_tiles, top_distance), dim=2)
        a0 = a0.squeeze()

        # Linear layer
        z1 = self.fc1(a0)

        # Calculate the L2 regularization term
        l2_reg_linear = 0.0
        for param in self.fc1.parameters():
            l2_reg_linear += torch.sum(param ** 2)

        # Nonlinear layer
        output = torch.sigmoid(z1)

        return output, l2_reg_conv, l2_reg_linear

In [15]:
demo_model = CHOWDER()

demo_model

CHOWDER(
  (conv): Conv1d(1, 1, kernel_size=(2048,), stride=(2048,))
  (fc1): Linear(in_features=5, out_features=1, bias=True)
)

## Setup Hyperparameters and Data Loader

In [16]:
# Define hyperparameters
BATCH_SIZE = 10
LR = 0.001
EPOCHS = 20
NUM_ENSEMBLE_MODELS = 10

In [17]:
class CustomDataset(Dataset):

    def __init__(self, data_tensor, target_tensor):
        self.data = data_tensor
        self.target = target_tensor

    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]
        return x, y

    def __len__(self):
        return len(self.data)

# Create custom datasets
train_dataset = CustomDataset(X_train_tensor, torch.Tensor(y_train))
val_dataset = CustomDataset(X_val_tensor, torch.Tensor(y_val))

# Create DataLoader
train_dl = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
val_dl = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)

In [18]:
# Free Up Memory
del X_train_tensor
del X_val_tensor
del y_train
del y_val

## Train Model and Validate

In [19]:
# Define loss function and optimizer
loss_function = nn.BCELoss()

# Create an ensemble of models
ensemble_models = []

for _ in range(NUM_ENSEMBLE_MODELS):

    # Initialize the model
    model = CHOWDER()

    # Define optimizer
    optimizer = optim.Adam(model.parameters(), lr=LR)
    ensemble_models.append((model, optimizer))

# Setup model counter
model_counter = 1

# Training and validation loops for each model in the ensemble
for model, optimizer in ensemble_models:

    print(f'\n ----- Model {model_counter} -----')
    model_counter += 1

    # Loop thorugh each epoch
    for epoch in range(EPOCHS):

        ## Training loop

        # Put model in train mode
        model.train()

        # Initialize loss, AUC and count
        total_loss = 0.0
        auroc_hist_train = 0.0
        total_count = 0.0

        # Train in batches
        for batch_x, batch_y in train_dl:

            batch_x_transformed = batch_x.unsqueeze(1)

            # Zero gradients
            optimizer.zero_grad()

            # Make predictions and get L2-norm from conv layer and linear layer
            pred, L2_conv, L2_linear = model(batch_x_transformed)

            # Calculate loss
            batch_y = batch_y.view(-1, 1) # Reshape batch_y from (10) to (10,1)
            loss = loss_function(pred, batch_y) + (0.1*L2_conv) + (0.001*L2_linear)

            # Calculate gradients
            loss.backward()

            # Make step in gradient descent
            optimizer.step()

            # Add to loss counter for this epoch
            total_loss += loss.item() * len(batch_y)
            total_count += len(batch_y)

            # Calculate AUC for batch
            auroc_hist_train += torchmetrics.AUROC(task="binary")(pred, batch_y).item() * len(batch_y)

        # Calculate loss and AUC per sample
        train_average_loss = total_loss / total_count
        train_average_auc = auroc_hist_train/ total_count


        ## Validation loop

        # Put model in evaluation mode
        model.eval()

        # Fix gradients (only using model to predict)
        with torch.no_grad():

            # Initialize loss, AUC and count
            total_loss = 0.0
            auroc_hist_val = 0.0
            total_count = 0.0

            # Validate in batches
            for batch_x, batch_y in val_dl:

                batch_x_transformed = batch_x.unsqueeze(1)

                # Make predictions
                val_pred, _, _ = model(batch_x_transformed)

                # Calculate loss
                batch_y = batch_y.view(-1, 1) # Reshape batch_y from (10) to (10,1)
                loss = loss_function(val_pred, batch_y)

                # Add to loss for this epoch
                total_loss += loss.item() * len(batch_y)
                total_count += len(batch_y)

                # Calculate AUC for batch
                auroc_hist_val += torchmetrics.AUROC(task="binary")(val_pred, batch_y).item() * len(batch_y)

        # Calculate loss and AUC per sample
        val_average_loss = total_loss / total_count
        val_average_auc =  auroc_hist_val / total_count

        # Print results from each epoch
        print(f"Epoch [{epoch+1}/{EPOCHS}] - Train Loss: {train_average_loss:.4f} Train AUC Score: {train_average_auc:.4f} \
        Val Loss: {val_average_loss:.4f} Val AUC Score: {val_average_auc:.4f}")


 ----- Model 1 -----
Epoch [1/20] - Train Loss: 29.2199 Train AUC Score: 0.4112         Val Loss: 24.9475 Val AUC Score: 0.4736
Epoch [2/20] - Train Loss: 24.5978 Train AUC Score: 0.4738         Val Loss: 22.3249 Val AUC Score: 0.6107
Epoch [3/20] - Train Loss: 17.4038 Train AUC Score: 0.4929         Val Loss: 7.2578 Val AUC Score: 0.5652
Epoch [4/20] - Train Loss: 6.8885 Train AUC Score: 0.5291         Val Loss: 6.7520 Val AUC Score: 0.4776
Epoch [5/20] - Train Loss: 5.0005 Train AUC Score: 0.4752         Val Loss: 4.7688 Val AUC Score: 0.5745
Epoch [6/20] - Train Loss: 3.1711 Train AUC Score: 0.5796         Val Loss: 1.8188 Val AUC Score: 0.5469
Epoch [7/20] - Train Loss: 3.6566 Train AUC Score: 0.5097         Val Loss: 2.9487 Val AUC Score: 0.3527
Epoch [8/20] - Train Loss: 2.0251 Train AUC Score: 0.5349         Val Loss: 1.2657 Val AUC Score: 0.5088
Epoch [9/20] - Train Loss: 1.5402 Train AUC Score: 0.5490         Val Loss: 1.0604 Val AUC Score: 0.3457
Epoch [10/20] - Train Loss: 

/usr/local/lib/python3.10/dist-packages/torchmetrics/utilities/prints.py:43: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)  # noqa: B028


Epoch [16/20] - Train Loss: 0.5720 Train AUC Score: 0.8203         Val Loss: 0.8738 Val AUC Score: 0.3259
Epoch [17/20] - Train Loss: 0.5497 Train AUC Score: 0.8957         Val Loss: 0.7765 Val AUC Score: 0.4737
Epoch [18/20] - Train Loss: 0.5376 Train AUC Score: 0.8973         Val Loss: 0.8226 Val AUC Score: 0.3646
Epoch [19/20] - Train Loss: 0.5284 Train AUC Score: 0.9184         Val Loss: 0.7995 Val AUC Score: 0.4090
Epoch [20/20] - Train Loss: 0.5170 Train AUC Score: 0.9184         Val Loss: 0.8124 Val AUC Score: 0.3770

 ----- Model 2 -----
Epoch [1/20] - Train Loss: 1.6864 Train AUC Score: 0.4939         Val Loss: 0.9000 Val AUC Score: 0.4650
Epoch [2/20] - Train Loss: 1.0606 Train AUC Score: 0.5109         Val Loss: 0.6864 Val AUC Score: 0.5200
Epoch [3/20] - Train Loss: 0.7704 Train AUC Score: 0.6188         Val Loss: 0.6648 Val AUC Score: 0.6534
Epoch [4/20] - Train Loss: 0.6748 Train AUC Score: 0.6995         Val Loss: 0.6318 Val AUC Score: 0.7098
Epoch [5/20] - Train Loss: 0

In [20]:
# After training all models, calculate the ensemble AUC

# Store predictions and labels
ensemble_predictions = []
val_labels = []

# Do not change gradients
with torch.no_grad():

    # Loop through each model
    for model, _ in ensemble_models:

        # Put model in evaluation mode
        model.eval()

        # Get predictions on the validation dataset
        predictions = []
        batch_labels = []  # Create a list to store labels for each batch

        for batch_x, batch_y in val_dl:

            batch_x_transformed = batch_x.unsqueeze(1)

            # Make predictions
            pred, _, _ = model(batch_x_transformed)
            # Save predictions
            predictions.append(pred)

            # Store val labels for this batch
            batch_labels.append(batch_y)

        # Concatenate the labels for this model
        batch_labels = torch.cat(batch_labels, dim=0)
        val_labels.append(batch_labels)

        predictions = torch.cat(predictions)
        ensemble_predictions.append(predictions)

# Concatenate all the validation labels
val_labels = torch.cat(val_labels, dim=0)

# Average the predictions from all models
ensemble_predictions = torch.stack(ensemble_predictions)
average_predictions = torch.mean(ensemble_predictions, dim=0)

# Calculate the AUC score based on the averaged predictions
average_auc = torchmetrics.AUROC(task="binary")(average_predictions, val_labels).item()
print(f"Ensemble AUC Score: {average_auc:.4f}")


Ensemble AUC Score: 0.2933


In [21]:
sys.exit()

NameError: ignored

## Test Model

In [ ]:
## Setup DataLoaders

# Create a dummy y for test set
y_test_dummy = torch.zeros(len(X_test_tensor),)

# Create custom datasets
test_dataset = CustomDataset(X_test_tensor, y_test_dummy)

# Create DataLoader
test_dl = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Merge them using ConcatDataset
dev_dataset = ConcatDataset([train_dataset, val_dataset])

# Create a DataLoader for the merged dataset
dev_dl = DataLoader(dev_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)

In [ ]:
# Free Up Memory
del X_test_tensor
del train_dataset
del val_dataset

In [ ]:
# Define loss function and optimizer
loss_function = nn.BCELoss()

# Create an ensemble of models
ensemble_models = []

for _ in range(NUM_ENSEMBLE_MODELS):

    # Initialize the model
    model = CHOWDER()

    # Define optimizer
    optimizer = optim.Adam(model.parameters(), lr=LR)
    ensemble_models.append((model, optimizer))

# Setup model counter
model_counter = 1

# Training loops for each model in the ensemble
for model, optimizer in ensemble_models:

    print(f'\n ----- Model {model_counter} -----')
    model_counter += 1

    # Loop thorugh each epoch
    for epoch in range(EPOCHS):

        ## Training loop

        # Put model in train mode
        model.train()

        # Initialize loss, AUC and count
        total_loss = 0.0
        auroc_hist_train = 0.0
        total_count = 0.0

        # Train in batches
        for batch_x, batch_y in dev_dl:

            # Add a dimension for channel numbers
            batch_x_transformed = batch_x.unsqueeze(1)

            # Zero gradients
            optimizer.zero_grad()

            # Make predictions and get L2-norm from conv layer
            pred, L2_term = model(batch_x_transformed)

            # Calculate loss
            batch_y = batch_y.view(-1, 1) # Reshape batch_y from (10) to (10,1)
            loss = loss_function(pred, batch_y) + (0.5*L2_term)

            # Calculate gradients
            loss.backward()

            # Make step in gradient descent
            optimizer.step()

            # Add to loss counter for this epoch
            total_loss += loss.item() * len(batch_y)
            total_count += len(batch_y)

            # Calculate AUC for batch
            auroc_hist_train += torchmetrics.AUROC(task="binary")(pred, batch_y).item() * len(batch_y)

        # Calculate loss and AUC per sample
        train_average_loss = total_loss / total_count
        train_average_auc = auroc_hist_train/ total_count

        # Print results from each epoch
        print(f"Epoch [{epoch+1}/{EPOCHS}] - Train Loss: {train_average_loss:.4f} Train AUC Score: {train_average_auc:.4f}")

In [ ]:
# After training all models, calculate the ensemble AUC

# Store predictions and labels
ensemble_predictions = np.zeros((len(test_dataset), len(ensemble_models)))

# Do not change gradients
with torch.no_grad():

    model_counter = 0

    # Loop through each model
    for model, _ in ensemble_models:

        # Put model in evaluation mode
        model.eval()

        # Get predictions on the validation dataset
        predictions = np.empty((0, 1))

        for batch_x, _ in test_dl:

            extra = 10 - len(batch_x)

            # If batch size is smaller than 10, pad rows in batch_x with 0s
            if extra > 0:
                pad_tensor = torch.zeros((extra,) + batch_x.shape[1:], dtype=batch_x.dtype)
                batch_x = torch.cat((batch_x, pad_tensor), dim=0)

            # Add a dimension for channel numbers
            batch_x_transformed = batch_x.unsqueeze(1)

            # Make predictions
            pred, _ = model(batch_x_transformed)

            # Save predictions
            pred_numpy = pred.numpy()
            predictions = np.concatenate((predictions, pred_numpy), axis=0)

            if extra > 0:
              predictions = predictions[:-extra]

        ensemble_predictions[:, model_counter] = predictions.squeeze()
        model_counter += 1

# Average the predictions from all models
average_prediction = np.mean(ensemble_predictions, axis=1)

In [ ]:
# Load metadata about each sample
data_path = '/content/drive/My Drive/Breast_Cancer_Detection/Data/'
df_test = pd.read_csv(data_path + "test_metadata.csv")

# Join sample ID metadata with probability prediction
CHOWDER_submission = pd.DataFrame( {"Sample ID": df_test["Sample ID"].values, "Target": average_prediction}).sort_values("Sample ID")

In [ ]:
def sanity_checks(submission):
    assert all(submission["Target"].between(0, 1)), "`Target` values must be in [0, 1]"
    assert submission.shape == (149, 2), "Your submission file must be of shape (149, 2)"
    assert list(submission.columns) == ["Sample ID", "Target",], "Your submission file must have columns `Sample ID` and `Target`"

sanity_checks(CHOWDER_submission)

In [ ]:
submission_path = '/content/drive/My Drive/Breast_Cancer_Detection/Predictions/'

CHOWDER_submission.to_csv(submission_path + "CHOWDER_submission_weights.csv", index=None)